In [1]:
from openai import OpenAI
from elasticsearch import Elasticsearch
import minsearch, json, requests
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
docs_json = docs_response.json()

In [ ]:
documents = []

for course in docs_json:
    course_name = course['course']
    
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        

In [ ]:
min_index = minsearch.Index(
    text_fields=['section', 'question', 'text'],
    keyword_fields=['course']
)

In [ ]:
min_index.fit(documents)

In [ ]:
boost = {'question': 3.0, 'text': 0.5}

retrieved = min_index.search(
    query=query,
    boost_dict=boost,
    filter_dict={'course':'data-engineering-zoomcamp'},
    num_results=5
)

In [2]:
elastic_client = Elasticsearch('http://localhost:9200')

## Elasticsearch

In [ ]:
def build_index(documents):
    index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
            }
        }
    }

    elastic_client.indices.create(index="course-questions", body=index_settings)

    for doc in tqdm(documents):
        elastic_client.index(index="course-questions", document=doc)

In [ ]:
build_index(documents)

In [3]:
def retriever(question_input):
    query_settings = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question_input,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    search_results = elastic_client.search(index="course-questions", body=query_settings)

    doc_results = []

    for doc in search_results['hits']['hits']:
        doc_results.append(doc['_source'])

    return doc_results

In [4]:
def prompt_constructor(question_input, context_input):
    prompt_template = """
    You are a course teaching assisstant. Answer this QUESTION based on the CONTEXT.
    
    QUESTION:
    {question}
    
    CONTEXT:
    {context}
    """.strip()

    context = ""

    for doc in context_input:
        context = context + f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"

    full_prompt = prompt_template.format(question= question_input, context= context)
    
    return full_prompt

In [5]:
def generate_answer(prompt):
    openai_client = OpenAI()
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages= [{'role':'user','content':prompt}]
    )

    return response.choices[0].message.content

In [6]:
def main(question):
    prompts = prompt_constructor(question, retriever(question))
    answer = generate_answer(prompts)

    return answer

In [7]:
question = input("Your question: ")
print()
print(main(question))

Your question:  how long this course?



The course is expected to take place over a period of approximately 4 months, as the Data-Engineering Zoomcamp usually runs from January to April each year.


In [ ]:
docker run -it \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:9.0.2